In [1]:
import numpy as np
import xarray as xr
import stackstac
import pystac_client
import planetary_computer
import rasterio as rio
import rioxarray as rxr
from urllib.request import urlretrieve

from dask.distributed import Client

In [2]:
client = Client()  # This starts the Dask client
print(client.dashboard_link)

/user/gbrencher/proxy/8787/status


In [3]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.27508544921875, 47.54687159892238],
            [-121.96128845214844, 47.54687159892238],
            [-121.96128845214844, 47.745787772920934],
            [-122.27508544921875, 47.745787772920934],
            [-122.27508544921875, 47.54687159892238],
        ]
    ],
}
bbox = rio.features.bounds(area_of_interest)

In [21]:
date_range = "2024-03-01/2024-04-01"

In [ ]:
stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

## grab S1 data

In [4]:
search = stac.search(
    bbox=bbox,
    datetime=date_range,
    collections=["sentinel-1-rtc"],
)

items = search.item_collection()
print(len(items))

9


In [16]:
ds = (
    stackstac.stack(
        items,
        resolution=50
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-1 uses 0 as nodata
)

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [17]:
# limit to morning (ascending orbit) acquisitons
ds = ds.where(ds.time.dt.hour > 11, drop=True)

In [18]:
# band dimension to data variable
ds = ds.to_dataset(dim='band')

## grab s2 data

In [28]:
search = stac.search(
    bbox=bbox,
    datetime=date_range,
    collections=["sentinel-2-l2a"],
    query={"eo:cloud_cover": {"lt": 25}},
)

items = search.item_collection()
print(len(items))

3


In [37]:
s2_ds = (
    stackstac.stack(
        items,
        resolution=50,
    )
    .where(lambda x: x > 0, other=np.nan)  # sentinel-2 uses 0 as nodata
)

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:364: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [35]:
# Convert the 'band' coordinate to data variables
s2_ds = s2_ds.to_dataset(dim='band')

In [36]:
s2_ds

<xarray.Dataset>
Dimensions:                                  (time: 3, y: 2238, x: 2238,
                                              band: 17)
Coordinates: (12/45)
  * time                                     (time) datetime64[ns] 2024-03-07...
    id                                       (time) <U54 'S2A_MSIL2A_20240307...
  * x                                        (x) float64 5e+05 ... 6.118e+05
  * y                                        (y) float64 5.301e+06 ... 5.189e+06
    s2:reflectance_conversion_factor         (time) float64 1.017 1.013 1.012
    s2:vegetation_percentage                 (time) float64 39.46 57.86 67.09
    ...                                       ...
    proj:shape                               (band) object [10980, 10980] ......
    proj:transform                           (band) object [10.0, 0.0, 499980...
    common_name                              (band) object None ... None
    center_wavelength                        (band) object None 0.443 ... None
    full_width_half_max                      (band) object None 0.027 ... None
    epsg                                     int64 32610
Dimensions without coordinates: band
Data variables: (12/17)
    AOT                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B01                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B02                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B03                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B04                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B05                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    ...                                       ...
    B12                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B8A                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    SCL                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    WVP                                      (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    visual                                   (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    preview                                  (time, y, x) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
Attributes:
    spec:        RasterSpec(epsg=32610, bounds=(499950, 5189200, 611850, 5301...
    crs:         epsg:32610
    transform:   | 50.00, 0.00, 499950.00|\n| 0.00,-50.00, 5301100.00|\n| 0.0...
    resolution:  50

## grab cop30 data

In [ ]:
search = stac.search(
    collections=["cop-dem-glo-30"],
    intersects=bbox
)

items = search.item_collection()
print(len(items))
    
data = []
for item in items:
    dem_path = planetary_computer.sign(item.assets['data']).href
    data.append(rxr.open_rasterio(dem_path))
cop30_da = merge_arrays(data)

## grab fcf data

In [19]:
def url_download(url, out_fp, overwrite = False):
    # check if file already exists
    if not exists(out_fp) or overwrite == True:
            urlretrieve(url, out_fp)
    # if already exists. skip download.
    else:
        print('file already exists, skipping')

In [20]:
def download_fcf(out_fp):
    # this is the url from Lievens et al. 2021 paper
    fcf_url = 'https://zenodo.org/record/3939050/files/PROBAV_LC100_global_v3.0.1_2019-nrt_Tree-CoverFraction-layer_EPSG-4326.tif'
    # download just forest cover fraction to out file
    url_download(fcf_url, out_fp)

<xarray.Dataset>
Dimensions:                                (time: 4, y: 5324, x: 8766, band: 2)
Coordinates: (12/38)
  * time                                   (time) datetime64[ns] 2024-03-06T1...
    id                                     (time) <U66 dask.array<chunksize=(4,), meta=np.ndarray>
  * x                                      (x) float64 3.736e+05 ... 8.119e+05
  * y                                      (y) float64 5.454e+06 ... 5.188e+06
    sar:looks_azimuth                      int64 1
    s1:product_timeliness                  <U8 'Fast-24h'
    ...                                     ...
    s1:datatake_id                         (time) <U6 dask.array<chunksize=(4,), meta=np.ndarray>
    proj:epsg                              int64 32610
    title                                  (band) <U41 dask.array<chunksize=(2,), meta=np.ndarray>
    description                            (band) <U173 dask.array<chunksize=(2,), meta=np.ndarray>
    raster:bands                           object {'nodata': -32768, 'data_ty...
    epsg                                   int64 32610
Dimensions without coordinates: band
Data variables:
    vh                                     (time, y, x) float64 dask.array<chunksize=(4, 1024, 1024), meta=np.ndarray>
    vv                                     (time, y, x) float64 dask.array<chunksize=(4, 1024, 1024), meta=np.ndarray>
Attributes:
    spec:        RasterSpec(epsg=32610, bounds=(373650, 5187950, 811950, 5454...
    crs:         epsg:32610
    transform:   | 50.00, 0.00, 373650.00|\n| 0.00,-50.00, 5454150.00|\n| 0.0...
    resolution:  50

In [ ]:
fcf_path ='../data/tmp/fcf_global.tif'
download_fcf(fcf_path)

In [ ]:
# open as dataArray and return
fcf = rxr.open_rasterio(fcf_path)

# clip first to avoid super long reproject processes
fcf = fcf.rio.clip_box(*aso_raster.rio.transform_bounds("EPSG:4326"))
# reproject FCF to match dataset
fcf = fcf.rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)
# remove band dimension as it only has one band
fcf = fcf.squeeze('band')
# if max is greater than 1 set to 0-1
if fcf.max() >= 1:
    fcf = fcf / 100

## combine datasets

In [6]:
ds = ds.persist()

In [7]:
median = ds.median(dim="time").compute()

2024-08-21 16:50:39,128 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 3.05 GiB -- Worker memory limit: 3.83 GiB
2024-08-21 16:50:39,326 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:33063 (pid=3961) exceeded 95% memory budget. Restarting...
2024-08-21 16:50:39,389 - distributed.nanny - WARNING - Restarting worker
Task exception was never retrieved
future: <Task finished name='Task-1112' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:2217> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py", line 2226, in wait
    raise AllExit()
distributed.client.AllExit
2024-08-2

In [ ]:
# set chunks
ds = ds.chunk({"x": 1024, "y": 1024, "time": -1})

In [9]:
client.close()